# Configurar Producto (Escribir en OPC UA)

Este script se conecta al PLC y modifica las variables de configuración del producto (Caja).

In [1]:
import asyncio
from asyncua import Client, ua

ENDPOINT = "opc.tcp://192.168.0.20:4840"
FOLDER_NAME = "MES"

async def find_node(client, folder_name, var_name, max_depth=5):
    """Busca recursivamente la carpeta y luego la variable."""
    async def walk(node, depth):
        if depth > max_depth: return None
        try: children = await node.get_children()
        except: return None
        
        for child in children:
            try: bname = (await child.read_browse_name()).Name
            except: continue
            
            if bname == folder_name:
                for var in await child.get_children():
                    if (await var.read_browse_name()).Name == var_name:
                        return var
            
            found = await walk(child, depth + 1)
            if found: return found
        return None
        
    return await walk(client.nodes.objects, 0)

async def write_opcua_multiple(variables_dict):
    """Se conecta al servidor y escribe múltiples valores a la vez."""
    async with Client(url=ENDPOINT) as client:
        for var_name, value in variables_dict.items():
            node = await find_node(client, FOLDER_NAME, var_name)
            if not node:
                print(f"❌ No se encontró la variable: {var_name}")
                continue
                
            # Asumimos que las dimensiones de caja son enteros (Int16 por defecto si hace falta o Inferencia)
            await node.write_value(value)
            print(f"✅ ¡Éxito! Variable '{var_name}' actualizada a: {value}")

In [2]:
box_type_val = 1
altura_val = 1
anchura_val = 1
largo_val = 1


In [3]:
# Crear el diccionario con los nombres de variables OPC UA y sus valores
variables_a_escribir = {
    "Box_Type": box_type_val,
    "Altura": altura_val,
    "Anchura": anchura_val,
    "Largo": largo_val
}

# Ejecutar la escritura
await write_opcua_multiple(variables_a_escribir)

Requested session timeout to be 3600000ms, got 30000ms instead


❌ No se encontró la variable: Box_Type


❌ No se encontró la variable: Altura


❌ No se encontró la variable: Anchura


❌ No se encontró la variable: Largo
